# Notes

Fourth iteration of the modeling:
- modulirized data processing
- model serialization

In [1]:
import os 
import re
import csv
import json
import math
from itertools import chain
from datetime import datetime

import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
plt.style.use('_mpl-gallery')

# Constants

In [2]:
SEQ_MAX_LEN = 600
NUM_CLASSES = 6
NUM_EXERCISES = 9
NUM_FLAG_BS = 2

In [3]:
dir_root = os.path.join('..')
dir_data_root = os.path.join(dir_root, 'data')
dir_exercises = os.path.join(dir_data_root, 'json', 'exercises_raw')
dir_exercises_test = os.path.join(dir_data_root, 'json', 'exercises_test')
dir_exercises_augmented = os.path.join(dir_data_root, 'json', 'exercises_augmented')
dir_patiens_sessions = os.path.join(dir_data_root, 'json', 'patients_sessions')

# Data prep

### Build training set

In [4]:
from utils.input import setup, exercise_to_input

In [5]:
setup(os.path.join('..'), SEQ_MAX_LEN, NUM_EXERCISES, NUM_FLAG_BS)

{'ROOT_PATH': '..',
 'ALL_REGIONS': ['frontal', 'orbital', 'oral'],
 'REGIONS': {'0_LefteyeMidbottom': 'orbital',
  '1_LefteyeMidtop': 'orbital',
  '2_LefteyeInnercorner': 'orbital',
  '3_LefteyeOutercorner': 'orbital',
  '4_LefteyebrowInner': 'frontal',
  '5_LefteyebrowCenter': 'frontal',
  '6_RighteyeMidbottom': 'orbital',
  '7_RighteyeMidtop': 'orbital',
  '8_RighteyeInnercorner': 'orbital',
  '9_RighteyeOutercorner': 'orbital',
  '10_RighteyebrowInner': 'frontal',
  '11_RighteyebrowCenter': 'frontal',
  '12_NoseTip': 'frontal',
  '13_MouthLowerlipMidbottom': 'oral',
  '14_MouthLeftcorner': 'oral',
  '15_MouthRightcorner': 'oral',
  '16_MouthUpperlipMidtop': 'oral',
  '17_ChinCenter': 'oral',
  '18_ForeheadCenter': 'frontal',
  '19_LeftcheekCenter': 'oral',
  '20_RightcheekCenter': 'oral'},
 'BASES': {'0_LefteyeMidbottom': '18_ForeheadCenter',
  '1_LefteyeMidtop': '18_ForeheadCenter',
  '2_LefteyeInnercorner': '18_ForeheadCenter',
  '3_LefteyeOutercorner': '18_ForeheadCenter',
  '4_

In [25]:
exercises_sources = [
    #dir_exercises_test
    dir_exercises,
    #dir_exercises_augmented
]

In [26]:
feature_engineering_setting = {
    'coordinates': True,
    'normalize_by_start': False,
    'normalize': False,
    'direction': False,
    'distance': False,
    're_base': True,
    'transformation_to_rebase': False,
    'extended_meta': True,
}

In [27]:
xslist_meta = list()
xslist_global = list()
xslist_frontal = list()
xslist_oral = list()
xslist_orbital = list()
yslist = list()

for exercise_source in exercises_sources:
    for file_name in os.listdir(exercise_source):
        file_path = os.path.join(exercise_source, file_name)

        if file_name == '.DS_Store': continue

        _xs_meta, _xs_global, _xs_frontal, _xs_oral, _xs_orbital, _ys = exercise_to_input(file_path, feature_engineering_setting)

        yslist.append(_ys)
        xslist_meta.append(_xs_meta)  
        xslist_global.append(_xs_global)
        xslist_frontal.append(_xs_frontal)
        xslist_oral.append(_xs_oral)
        xslist_orbital.append(_xs_orbital)
            
ys = np.array(yslist)
xs_meta = np.array(xslist_meta)   
xs_global = np.array(xslist_global) 
xs_frontal = np.array(xslist_frontal) 
xs_oral = np.array(xslist_oral) 
xs_orbital = np.array(xslist_orbital) 

print(ys.shape)
print(xs_meta.shape)
print(xs_global.shape)
print(xs_frontal.shape)
print(xs_oral.shape)
print(xs_orbital.shape)

(2146,)
(2146, 54)
(2146, 126, 600)
(2146, 36, 600)
(2146, 42, 600)
(2146, 48, 600)


## Modeling

In [28]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from sklearn.utils import class_weight

In [29]:
def get_dnn(inputLayer): 
    m = Dense(4, activation="relu")(inputLayer)
    m = Model(inputs=inputLayer, outputs=m)

    return m

In [30]:
def get_cnn(inputLayer):
    chanDim = -1
    
    m = Conv1D(16, 3, padding='same', activation='relu')(inputLayer)
    m = BatchNormalization(axis=chanDim)(m)
    m = MaxPooling1D((2))(m)
    m = Conv1D(32, 3, padding='same', activation='relu')(m)
    m = BatchNormalization(axis=chanDim)(m)
    m = MaxPooling1D((2))(m)
    m = Conv1D(64, 3, padding='same', activation='relu')(m)
    m = BatchNormalization(axis=chanDim)(m)
    m = MaxPooling1D((2))(m)
    m = Conv1D(64, 3, padding='same', activation='relu')(m)
    m = BatchNormalization(axis=chanDim)(m)
    m = MaxPooling1D((2))(m)
    m = Flatten()(m)
    m = Dropout(0.5)(m)
    m = Dense(128, activation="relu")(m)
    m = Model(inputs=inputLayer, outputs=m)

    return m

In [31]:
def get_model():
    input_meta = Input(shape=xs_meta.shape[1:])
    model_meta = get_dnn(input_meta)
    
    input_global = Input(shape=xs_global.shape[1:])
    model_global = get_cnn(input_global)
    
    input_frontal = Input(shape=xs_frontal.shape[1:])
    model_frontal = get_cnn(input_frontal)  

    input_oral = Input(shape=xs_oral.shape[1:])
    model_oral = get_cnn(input_oral)  
    
    input_orbital = Input(shape=xs_orbital.shape[1:])
    model_orbital = get_cnn(input_orbital)  
    
    
    model_contatenate = concatenate([
        model_meta.output, 
        model_global.output,
        model_frontal.output,
        model_oral.output,
        model_orbital.output,
    ])
    
    model_contatenate = Dense(32, activation="relu")(model_contatenate)
    model_contatenate = Dense(6, activation="softmax")(model_contatenate)
        
    model = Model(inputs=[
        model_meta.input,
        model_global.input,
        model_frontal.input,
        model_oral.input,
        model_orbital.input
    ], outputs=model_contatenate)

    model.compile(
        loss="sparse_categorical_crossentropy", 
        optimizer=Adam(learning_rate=1e-3, decay=1e-3 / 200),
        metrics=['accuracy']
    )
    
    
    return model

In [32]:
test_model = get_model()
test_model.summary()

Model: "model_71"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_57 (InputLayer)          [(None, 126, 600)]   0           []                               
                                                                                                  
 input_58 (InputLayer)          [(None, 36, 600)]    0           []                               
                                                                                                  
 input_59 (InputLayer)          [(None, 42, 600)]    0           []                               
                                                                                                  
 input_60 (InputLayer)          [(None, 48, 600)]    0           []                               
                                                                                           

 batch_normalization_178 (Batch  (None, 31, 64)      256         ['conv1d_178[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_182 (Batch  (None, 9, 64)       256         ['conv1d_182[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_186 (Batch  (None, 10, 64)      256         ['conv1d_186[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_190 (Batch  (None, 12, 64)      256         ['conv1d_190[0][0]']             
 Normaliza

                                                                                                  
 concatenate_11 (Concatenate)   (None, 516)          0           ['dense_77[0][0]',               
                                                                  'dense_78[0][0]',               
                                                                  'dense_79[0][0]',               
                                                                  'dense_80[0][0]',               
                                                                  'dense_81[0][0]']               
                                                                                                  
 dense_82 (Dense)               (None, 32)           16544       ['concatenate_11[0][0]']         
                                                                                                  
 dense_83 (Dense)               (None, 6)            198         ['dense_82[0][0]']               
          

In [33]:
print(set(ys))

{0, 1, 2, 3, 4, 5}


### K-fold evaluation

In [34]:
k = 5
k_limit = 5
train = 0.8
val = 0.2
test = 0.2

In [35]:
VERBOSE = 0
EPOCHS = 400
BATCH_SIZE = 8

In [36]:
from sklearn.model_selection import KFold

In [37]:
def get_k_indx(k, n):

    k_fold = KFold(n_splits=k)
    train_ = []
    val_ = []
    test_ = []
    indx = []

    for train_indices, test_indices in k_fold.split(ys):
        n_k = len(train_indices)
        val_split = int(n_k * train)
        indx.append([train_indices[:val_split],train_indices[val_split + 1:], test_indices])
    
    return indx

In [38]:
%matplotlib inline

In [ ]:
indxs = get_k_indx(k, len(ys))
models = []

for i in range(k_limit):
    train_indx, val_indx, test_indx  = indxs[i]
    xs_meta_i = xs_meta[train_indx]
    xs_meta_i_val = xs_meta[val_indx]
    xs_meta_i_test = xs_meta[test_indx]
    
    xs_global_i = xs_global[train_indx]
    xs_global_i_val = xs_global[val_indx]
    xs_global_i_test = xs_global[test_indx]
    
    xs_frontal_i = xs_frontal[train_indx]
    xs_frontal_i_val = xs_frontal[val_indx]
    xs_frontal_i_test = xs_frontal[test_indx]
    
    xs_oral_i = xs_oral[train_indx]
    xs_oral_i_val = xs_oral[val_indx]
    xs_oral_i_test = xs_oral[test_indx]
    
    xs_orbital_i = xs_orbital[train_indx]
    xs_orbital_i_val = xs_orbital[val_indx]
    xs_orbital_i_test = xs_orbital[test_indx]
    
    ys_i = ys[train_indx]
    ys_i_val = ys[val_indx]
    ys_i_test = ys[test_indx]
    
    
    class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                 classes = np.unique(ys_i),
                                                 y=ys_i)
    available_classes = np.unique(ys_i)
    weight_training_classes = {
        0: 0,
        1: 0,
        2: 0,
        3: 0,
        4: 0,
        5: 0,
    }
    
    for indx, value in enumerate(class_weights):
        weight_training_classes[available_classes[indx]] = value

    #print(weight_training_classes)

    model = get_model()

    model_callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=20)
    ]
    
    hist = model.fit(
        x=[
            xs_meta_i, 
            xs_global_i, 
            xs_frontal_i,
            xs_oral_i,
            xs_orbital_i], y=ys_i, 
        validation_data=([
            xs_meta_i_val,
            xs_global_i_val,
            xs_frontal_i_val,
            xs_oral_i_val,
            xs_orbital_i_val], ys_i_val),
        batch_size=BATCH_SIZE, 
        epochs=EPOCHS,
        class_weight=weight_training_classes,
        #callbacks=model_callbacks,
        verbose=VERBOSE)
    
    models.append(model)
    
    # visualizing losses and accuracy
    train_loss = hist.history['loss']
    val_loss   = hist.history['val_loss']
    train_acc  = hist.history['accuracy']
    val_acc    = hist.history['val_accuracy']
    xc         = range(len(hist.history['loss']))    
    
    plt.figure()
    plt.title(f'Run {i}')
    plt.plot(xc, train_loss, color='red')
    plt.plot(xc, val_loss, color='pink')
    plt.plot(xc, train_acc, color='blue')
    plt.plot(xc, val_acc, color='cyan')
    
    y_pred = model.predict([
        xs_meta_i_test,
        xs_global_i_test,
        xs_frontal_i_test,
        xs_oral_i_test,
        xs_orbital_i_test],verbose=0)
    y_pred_bool = np.argmax(y_pred, axis=1)

    print(classification_report(ys_i_test, y_pred_bool))

2024-01-22 10:44:58.513358: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-01-22 10:45:54.124765: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-01-22 12:04:30.813034: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


              precision    recall  f1-score   support

           0       0.80      0.84      0.82       194
           1       0.79      0.56      0.66        75
           2       0.69      0.76      0.73        68
           3       0.85      0.48      0.61        23
           4       0.68      0.77      0.72        22
           5       0.61      0.77      0.68        48

    accuracy                           0.75       430
   macro avg       0.74      0.70      0.70       430
weighted avg       0.76      0.75      0.75       430



2024-01-22 12:04:50.765168: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-01-22 12:05:54.540544: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


# Model Serialization 

In [ ]:
serialize = False
best_model = 0

In [ ]:
dir_models = os.path.join(dir_root, 'models')
dir_model = os.path.join(dir_models, 'type4-with-fe')

In [ ]:
if serialize:
    model = models[best_model]
    model.save(dir_model)